In [7]:
import pandas as pd
import numpy as np
import bentoml

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

## Q1 solution

In [8]:
!bentoml --version

bentoml, version 1.0.5


### Data preparation

In [9]:
data = 'CreditScoring.csv'
df = pd.read_csv(data)

In [10]:
df.columns = df.columns.str.lower()

status_values = {
    1: 'ok',
    2: 'default',
    0: 'unk'
}

df.status = df.status.map(status_values)

home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [11]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [12]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
X_test = dv.transform(test_dicts)

### Random forest

In [ ]:
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(X_train, y_train)

In [ ]:
bentoml.sklearn.save("credit_risk_model-1", rf,     
        custom_objects={
        'dictVectorizer': dv
        }
    )



### BentoML

In [ ]:
import numpy as np

import bentoml
from bentoml.io import JSON

model_ref = bentoml.sklearn.get("credit_risk_model:24jb4vcqmsawhlg6")
dv = model_ref.custom_objects['dictVectorizer']

model_runner = model_ref.to_runner()

svc = bentoml.Service("credit_risk_classifier", runners=[model_runner])

Test

In [ ]:
application_data={
  "name": "Tim",
  "age": 37,
  "country": "US",
  "rating": 3.14
}

## XGboost

In [13]:
from pydantic import BaseModel

In [ ]:
class UserProfile(BaseModel):
    name: str
    age: int
    country: str
    rating: float


In [14]:
import xgboost as xgb

In [15]:
dtrain = xgb.DMatrix(X_train, label=y_train)

In [16]:
xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

In [18]:
bentoml.xgboost.save("credit_risk_model-XG", model,     
        custom_objects={
        'dictVectorizer': dv
        }
    )

The "bentoml.xgboost.save" method is being deprecated. Use "bentoml.xgboost.save_model" instead
converting 'credit_risk_model-XG' to lowercase: 'credit_risk_model-xg'


Model(tag="credit_risk_model-xg:dzaujxssuwwpflg6", path="/Users/jainendra/bentoml/models/credit_risk_model-xg/dzaujxssuwwpflg6/")

## Q2 solution

In [19]:
!ls -ltr /Users/jainendra/bentoml/models/credit_risk_model-xg/dzaujxssuwwpflg6/ 

total 248
-rw-r--r--  1 jainendra  staff     716 Oct 23 13:04 custom_objects.pkl
-rw-r--r--  1 jainendra  staff     349 Oct 23 13:04 model.yaml
-rw-r--r--  1 jainendra  staff  117938 Oct 23 13:04 saved_model.ubj


## Q3 solution in server.py

## Q4 , Q5, Q6 solution in server_numarray.py